In [ ]:
!pip install -U "langchain>=0.3.12" \
               "langchain-core>=0.3.30" \
               "langchain-google-genai>=2.0.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.1/102.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.0/475.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.7/343.7 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.1.0
    Uninstalling langchain-core-1.1.0:
      Successfully uninstalled langchain-core-1.1.0
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
  Attempting uninstall: langchain
    Found existing installation: langchain 1.1.0
    Uninstalling langchain-1.1.0:
      Successfully uninstalled langchain-1.1.0
ERROR: pip's

## Package installation

In [ ]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
import requests
#from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain.tools import tool
import os

rom google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
gemini_api_key = userdata.get("GOOGLE_API_KEY")
# Get API keys from environment variables
f
if not gemini_api_key:
    print("Warning: GEMINI_API_KEY not found. Gemini model will not run.")


In [ ]:
# -----------------------------
# 1. Weather API function
# -----------------------------
GEOCODING_API_URL = "https://geocoding-api.open-meteo.com/v1/search"
FORECAST_URL = "https://api.open-meteo.com/v1/forecast"

In [ ]:
@tool
def get_weather(city: str) -> str:
    """Get current weather for a given city."""
    # Geocode the city
    r = requests.get(GEOCODING_API_URL, params={"name": city, "count": 1}, timeout=15)
    r.raise_for_status()
    data = r.json()
    if not data.get("results"):
        return f"Error: could not find location for {city}"
    lat = data["results"][0]["latitude"]
    lon = data["results"][0]["longitude"]

    # Fetch weather
    w = requests.get(
        FORECAST_URL,
        params={"latitude": lat, "longitude": lon, "current_weather": True},
        timeout=15,
    )
    w.raise_for_status()
    weather = w.json().get("current_weather")
    return f"City: {city}, Lat: {lat}, Lon: {lon}, Current Weather: {weather}"

In [ ]:
# -----------------------------
# 3. Define the Chat Model
# -----------------------------
model = ChatGoogleGenerativeAI(
        #model="gemini-2.0-flash-lite-001",
        model="gemini-2.5-flash-lite",
        google_api_key=gemini_api_key,
        temperature=0.3 )

# -----------------------------
# 4. Create the Agent using create_agent
# -----------------------------

# class ProductReview(BaseModel):
#     resposne: str | None = Field(description="The weather information")


agent = create_agent(
    model=model,
    tools=[get_weather],  # pass the decorated tool directly

)

In [ ]:
user_input = "What is the weather in Tokyo?"
response = agent.invoke( {"messages": [{"role": "user", "content":user_input}]}
)

* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite
Please retry in 29.870102874s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 20
}
, retry_delay {
  seconds: 29
}
].
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite
Please retry in 27.787217419s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage

KeyboardInterrupt: 

In [ ]:


print("\n--- Final Agent Response ---\n")
#print(response)
print(response['messages'][-1].content)
#print(response["output_text"])
#print(response["structured_response"])




--- Final Agent Response ---

The weather in Tokyo is 5.2 degrees. The windspeed is 2.7 and the winddirection is 337.


In [ ]:
# ----------------------------------------------
# 1️ Imports
# ----------------------------------------------
import requests
from langchain.tools import tool
from langchain.agents import create_agent

# ----------------------------------------------
# 2️⃣ Currency Exchange Tool (converted)
# ----------------------------------------------
EXCHANGE_API_URL = "https://open.er-api.com/v6/latest/"

@tool
def currency_exchange(query: str) -> str:
    """
    Get exchange rate between two currencies.
    Expected format: 'USD to INR' or 'EUR to JPY'
    """
    # Parse input
    if " to " not in query:
        return "Error: format must be '<BASE> to <TARGET>'"

    base, target = query.split(" to ")
    base = base.strip().upper()
    target = target.strip().upper()

    # Fetch exchange API data
    r = requests.get(EXCHANGE_API_URL + base, timeout=15)
    if r.status_code != 200:
        return "Error: failed to retrieve exchange rates."

    data = r.json()
    if data.get("result") != "success":
        return "Error: exchange API returned failure."

    rate = data["rates"].get(target)
    if not rate:
        return f"Error: target currency '{target}' not found."

    # Produce nice formatted result
    return f"1 {base} = {rate} {target}"

In [ ]:
# -----------------------------
# 3. Define the Chat Model
# -----------------------------
model = ChatGoogleGenerativeAI(
        #model="gemini-2.0-flash-lite-001",
        model="gemini-2.5-flash-lite",
        google_api_key=gemini_api_key,
        temperature=0.3 )

# ----------------------------------------------
# 4️. Create Agent with Currency Tool
# ----------------------------------------------
agent = create_agent(
    model=model,
    tools=[currency_exchange],
)

In [ ]:

# ----------------------------------------------
# 5️. Run Agent
# ----------------------------------------------
user_question = "How much is USD to INR?"
response = agent.invoke({"messages": [{"role": "user", "content":user_question}]})

print("\n--- Final Agent Response ---\n")
print(response)



--- Final Agent Response ---

{'messages': [HumanMessage(content='How much is USD to INR?', additional_kwargs={}, response_metadata={}, id='8f3f48e1-1687-4c2f-aa41-a274dab2d664'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'currency_exchange', 'arguments': '{"query": "USD to INR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019af9c9-4716-7923-b767-69ea9c9e75c1-0', tool_calls=[{'name': 'currency_exchange', 'args': {'query': 'USD to INR'}, 'id': 'f4bd5f35-211a-4486-bf0b-0dc70f06d39d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 64, 'output_tokens': 17, 'total_tokens': 81, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='1 USD = 90.025086 INR', name='currency_exchange', id='f2e098e8-5ca2-4153-b309-3bd98fe64e19', tool_call_id='f4bd5f35-211a-4486-bf0b-0dc70f06d39d'), A

In [ ]:
# ======================================================
# 1️. Imports
# ======================================================
import requests
from langchain.tools import tool
from langchain.agents import create_agent


# ======================================================
# 2. Weather Tool
# ======================================================
GEOCODING_API_URL = "https://geocoding-api.open-meteo.com/v1/search"
FORECAST_URL = "https://api.open-meteo.com/v1/forecast"

@tool
def get_weather(city: str) -> str:
    """Get current weather for a given city."""
    r = requests.get(GEOCODING_API_URL, params={"name": city, "count": 1}, timeout=15)
    r.raise_for_status()
    data = r.json()

    if not data.get("results"):
        return f"Error: No location found for {city}"

    lat = data["results"][0]["latitude"]
    lon = data["results"][0]["longitude"]

    w = requests.get(
        FORECAST_URL,
        params={"latitude": lat, "longitude": lon, "current_weather": True},
        timeout=15,
    )
    w.raise_for_status()
    weather = w.json().get("current_weather")

    return f"City: {city}, Latitude: {lat}, Longitude: {lon}, Current Weather: {weather}"


# ======================================================
# 3️ Currency Exchange Tool
# ======================================================
EXCHANGE_API_URL = "https://open.er-api.com/v6/latest/"

@tool
def currency_exchange(query: str) -> str:
    """
    Get exchange rate between two currencies.
    Input format: 'USD to INR' or 'EUR to JPY'
    """
    if " to " not in query:
        return "Error: format must be '<BASE> to <TARGET>'"

    base, target = query.split(" to ")
    base = base.strip().upper()
    target = target.strip().upper()

    r = requests.get(EXCHANGE_API_URL + base, timeout=15)
    if r.status_code != 200:
        return "Error: Failed to retrieve exchange rates."

    data = r.json()
    if data.get("result") != "success":
        return "Error: Exchange API returned failure."

    rate = data["rates"].get(target)
    if not rate:
        return f"Error: Currency '{target}' not found."

    return f"1 {base} = {rate} {target}"


# ======================================================
# 4️ LLM Model
# ======================================================
model = ChatGoogleGenerativeAI(
        #model="gemini-2.0-flash-lite-001",
        model="gemini-2.5-flash-lite",
        google_api_key=gemini_api_key,
        temperature=0.3 )



# ======================================================
# 5️ Create Multi-Tool Agent
# ======================================================
agent = create_agent(
    model=model,
    tools=[get_weather, currency_exchange],
)


# ======================================================
# 6️ Test Agent with Sample Queries
# ======================================================
# print("\n--- Weather Example ---")
# user_question = "What is the weather in Tokyo?"
# print(agent.invoke({"messages": [{"role": "user", "content":user_question}]}))


# print("\n--- Currency Example ---")
# user_question = "Convert USD to INR"
# print(agent.invoke({"messages": [{"role": "user", "content":user_question}]}))

# print("\n--- Mixed Example ---")
user_question = "If 100 USD is converted to INR, how much will I get?"
print(agent.invoke({"messages": [{"role": "user", "content":user_question}]}))


{'messages': [HumanMessage(content='If 100 USD is converted to INR, how much will I get?', additional_kwargs={}, response_metadata={}, id='bda30b82-d9b9-4248-b168-9def1c9dc6ce'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'currency_exchange', 'arguments': '{"query": "USD to INR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019af9cd-eb40-75d1-8402-7cb6773c353d-0', tool_calls=[{'name': 'currency_exchange', 'args': {'query': 'USD to INR'}, 'id': '11d66971-ddcb-479b-a1c2-9510d76cd2d4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 114, 'output_tokens': 17, 'total_tokens': 131, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='1 USD = 90.025086 INR', name='currency_exchange', id='cfae0f83-df15-44a5-bde3-47fc35e55d5d', tool_call_id='11d66971-ddcb-479b-a1c2-9510d76cd2d4'), A

In [ ]:

# Get API keys from environment variables
from google.colab import userdata
gemini_api_key = userdata.get("GOOGLE_API_KEY")
if not gemini_api_key:
    print("Warning: GEMINI_API_KEY not found. Gemini model will not run.")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
import os
from langchain_google_genai import ChatGoogleGenerativeAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "Explain {topic} in simple words for a {age}-year-old.")
])

model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite-001",
        google_api_key=gemini_api_key,
        temperature=0.3 )

# This is an LCEL chain
chain = prompt | model

# Now you can invoke it with two variables
llm_response = chain.invoke({"topic": "LangChain Expression Language", "age": 10})


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# 1. Define the model, prompt, and parser
model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite-001",
        google_api_key=gemini_api_key,
        temperature=0.3 )

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{input}")
])

parser = StrOutputParser()

# 2. Create the chain
chain = prompt | model | parser

# 3. Invoke the chain to get a clean string result
result = chain.invoke({'input': 'What is LangChain Expression Language?'})
print(result)



LangChain Expression Language (LCEL) is a declarative way to define chains in LangChain. It allows you to build complex chains from simpler components in a more readable and maintainable way. Think of it as a way to "wire" together different LangChain components (like models, prompts, and output parsers) to create a complete application.

Here's a breakdown of what makes LCEL important and how it works:

**Key Features and Benefits of LCEL:**

*   **Declarative:** You describe *what* you want to do, not *how* to do it. This makes your code more concise and easier to understand.
*   **Composability:** You can easily combine smaller chains into larger, more complex ones. This promotes code reuse and modularity.
*   **Asynchronous by Default:** LCEL chains are designed to be asynchronous, meaning they can handle multiple requests concurrently, improving performance.
*   **Streaming Support:** LCEL supports streaming, allowing you to receive results from LLMs as they are generated, providi

In [ ]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory


# Memory store for different students/sessions
store = {}

def get_memory(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# ---- 1. ChatPromptTemplate ----
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "Write a 5-line summary on: {text}")
])

# ---- 2. Model and Parser ----
model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite-001",

        google_api_key=gemini_api_key,
        temperature=0.3 )
parser = StrOutputParser()

# ---- 3. LCEL Chain ----
chain = prompt | model | parser

# ---- 4. Add Memory Wrapper ----
chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_memory,
    input_messages_key="text",      # user message variable
    history_messages_key="history"  # internal memory key
)

# ---- 5. Run ----
session_id = "student_123"

response1 = chain_with_memory.invoke(
    {"text": "Explain how photosynthesis works."},
    config={"configurable": {"session_id": session_id}}
)

response2 = chain_with_memory.invoke(
    {"text": "Summarize your previous explanation in simpler words."},
    config={"configurable": {"session_id": session_id}}
)

print(response1)
print(response2)


Photosynthesis is the process plants use to convert light energy into chemical energy.  Plants absorb sunlight, water, and carbon dioxide.  Chlorophyll, a pigment in chloroplasts, captures the sunlight.  This energy drives a reaction that transforms water and CO2 into glucose (sugar) for food.  Oxygen is released as a byproduct of this crucial process.
Okay, here's a 5-line summary of how to summarize something in simpler words:

1.  **Understand the original:** Read or listen to the explanation carefully.
2.  **Identify the main points:** Figure out the most important ideas.
3.  **Simplify the language:** Use easier words and shorter sentences.
4.  **Keep it concise:** Condense the information into a shorter version.
5.  **Check for accuracy:** Make sure your summary still reflects the original meaning.


In [ ]:
from langchain_community.chat_message_histories import InMemoryChatMessageHistory

history = InMemoryChatMessageHistory()
history.add_user_message("Hello")
history.add_ai_message("Hi there!")

print(history.messages)


ImportError: cannot import name 'InMemoryChatMessageHistory' from 'langchain_community.chat_message_histories' (/usr/local/lib/python3.12/dist-packages/langchain_community/chat_message_histories/__init__.py)

In [ ]:
session_id = "student_1"

response1 = chain_with_memory.invoke(
    {"text": "Explain gravity."},
    config={"configurable": {"session_id": session_id}}
)

response2 = chain_with_memory.invoke(
    {"text": "Summarize your previous explanation."},
    config={"configurable": {"session_id": session_id}}
)

print(response1)
print(response2)


Please provide the text you would like me to summarize. I need the text "Explain gravity" to be able to write a 5-line summary about it.


## Demo1: Weather Agent

In [ ]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [ ]:
import os, requests

GEOCODING_API_URL = "https://geocoding-api.open-meteo.com/v1/search"
FORECAST_URL = "https://api.open-meteo.com/v1/forecast"

def get_weather(city: str, country: str | None = None) -> dict:
    # 1) geocode
    params = {"name": city, "count": 1}
    r = requests.get(GEOCODING_API_URL, params=params, timeout=15)
    r.raise_for_status()
    data = r.json()
    if not data.get("results"):
        return {"error": f"could not find location for {city}"}
    lat = data["results"][0]["latitude"]
    lon = data["results"][0]["longitude"]


    # 2) weather
    w = requests.get(
        FORECAST_URL,
        params={"latitude": lat, "longitude": lon, "current_weather": True},
        timeout=15,
    )
    w.raise_for_status()
    return {
        "city": city,
        "coords": {"lat": lat, "lon": lon},
        "current_weather": w.json().get("current_weather"),
    }

In [ ]:
print(get_weather("tokyo"))


[{'id': 1850147, 'name': 'Tokyo', 'latitude': 35.6895, 'longitude': 139.69171, 'elevation': 44.0, 'feature_code': 'PPLC', 'country_code': 'JP', 'admin1_id': 1850144, 'timezone': 'Asia/Tokyo', 'population': 9733276, 'country_id': 1861060, 'country': 'Japan', 'admin1': 'Tokyo'}]
{'city': 'tokyo', 'coords': {'lat': 35.6895, 'lon': 139.69171}, 'current_weather': {'time': '2025-12-03T10:30', 'interval': 900, 'temperature': 9.7, 'windspeed': 7.2, 'winddirection': 333, 'is_day': 0, 'weathercode': 0}}


In [ ]:
# Define a simple chain template for easy switching between models
def create_roles_chain(model):
    prompt = ChatPromptTemplate.from_template("{input}")
    return prompt | model | StrOutputParser()

In [ ]:
def gpt4o_mini_agent(user_input):
    # First, let GPT decide what to do
    # This agent will check the intent of the user
    # This agent will extract city name from the user query
    query_prompt = f"""You are an intelligent assistant with access to a weather API.

    If the user asks anything about the weather in a city, respond with:
    "CALL_WEATHER_API: <city-name>"

    Otherwise, answer normally.

    User input: "{user_input}"
    """

    print(query_prompt)

    gemini_model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite-001",
        google_api_key=gemini_api_key,
        temperature=0.3 )

    print("---   Response ---")

    assistant_reply = create_roles_chain(gemini_model).invoke({"input": query_prompt})
    print(f"[Assistant Thought] {assistant_reply}")

    # If GPT indicates to call the API
    if assistant_reply.startswith("CALL_WEATHER_API:"):
        city_name = assistant_reply.replace("CALL_WEATHER_API:", "").strip()

        # Step 1: Get latitude and longitude
        api_result = get_weather(city_name)

        # Step 3: Let Model (GPT/Gemini) format the final answer nicely
        final_prompt = f"The weather information for {city_name} is: {api_result}. Please compose a friendly response to the user."

        gemini_model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite-001",
        google_api_key=gemini_api_key,
        temperature=0.3 )

        print("---   Response [Final]---")
        final_answer = create_roles_chain(gemini_model).invoke({"input": final_prompt})

        return final_answer
    else:
        return assistant_reply

In [ ]:
print(gpt4o_mini_agent("What is the weather in tokyo"))






You are an intelligent assistant with access to a weather API.

    If the user asks anything about the weather in a city, respond with:
    "CALL_WEATHER_API: <city-name>"

    Otherwise, answer normally.

    User input: "What is the weather in tokyo"
    
---   Response ---
[Assistant Thought] CALL_WEATHER_API: tokyo

---   Response [Final]---
Okay, here's a friendly response based on the Tokyo weather information:

"Good morning! The weather in Tokyo right now is looking a bit chilly. It's currently 9.6°C (about 49°F) with a wind speed of 7.6 km/h. The wind is coming from the northwest (329 degrees). It's also nighttime, so no sunshine! Have a warm and cozy day!"



## Demo2: Currency exchange

In [ ]:
import requests

EXCHANGE_API_URL = "https://open.er-api.com/v6/latest/"

def call_exchange_api(base: str, target: str) -> dict:
    r = requests.get(EXCHANGE_API_URL + base.upper(), timeout=15)
    if r.status_code != 200:
        return {"error": "could not retrieve exchange rates"}
    data = r.json()
    if data.get("result") != "success":
        return {"error": "exchange service returned failure"}
    rate = data["rates"].get(target.upper())
    if not rate:
        return {"error": f"target {target} not found"}
    return {
        "base": base.upper(),
        "target": target.upper(),
        "rate": rate,
        "formatted": f"1 {base.upper()} = {rate} {target.upper()}",
    }

In [ ]:
def gpt4o_mini_currency_agent(user_input):
    query_prompt = f"""You are an assistant with access to a currency exchange API.

    If the user asks about exchanging currency, respond exactly:
    "CALL_EXCHANGE_API: <base-currency-code> to <target-currency-code>"

    Otherwise, answer normally.

    User input: "{user_input}"
    """

    gemini_model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite-001",
        google_api_key=gemini_api_key,
        temperature=0.3 )

    print("---   Response ---")

    assistant_reply = create_roles_chain(gemini_model).invoke({"input": query_prompt})

    print(f"[Assistant Thought] {assistant_reply}")


    if assistant_reply.startswith("CALL_EXCHANGE_API:"):
        parts = assistant_reply.replace("CALL_EXCHANGE_API:", "").strip().split(" to ")
        if len(parts) == 2:
            base_currency = parts[0]
            target_currency = parts[1]
            api_result = call_exchange_api(base_currency, target_currency)
            final_prompt = f"The exchange rate is: {api_result}. Please compose a friendly detailed response."
        else:
            return "Sorry, I couldn't understand the currency pair."
    else:
        return assistant_reply

    gemini_model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite-001",
        google_api_key=gemini_api_key,
        temperature=0.3 )

    print("---   Response [Final]---")
    final_response = create_roles_chain(gemini_model).invoke({"input": final_prompt})

    return final_response

In [ ]:
user_question = "How much is USD to INR?"


answer = gpt4o_mini_currency_agent(user_question)
print(answer)

---   Response ---
[Assistant Thought] CALL_EXCHANGE_API: USD to INR

---   Response [Final]---
Okay, I've got the exchange rate information! Let's break it down:

You've provided me with the following:

*   **Base Currency:** USD (United States Dollar)
*   **Target Currency:** INR (Indian Rupee)
*   **Exchange Rate:** 89.941202
*   **Formatted Representation:** "1 USD = 89.941202 INR"

**What this means in plain English:**

This tells us how much Indian Rupees (INR) you would get for one United States Dollar (USD).  Specifically, according to this information, **1 US Dollar is currently worth 89.941202 Indian Rupees.**

**Let's illustrate with a few examples:**

*   If you have $10 USD, you could exchange it for approximately 899.41 INR (10 USD * 89.941202 INR/USD).
*   If you have $100 USD, you could exchange it for approximately 8994.12 INR (100 USD * 89.941202 INR/USD).

**Important Considerations:**

*   **Real-time Fluctuations:** Exchange rates are constantly changing. This rate

# Demo 3: Master Agent

In [ ]:
import os, json

def route_intent(user_text: str) -> dict:

    query_prompt = f"""
          You are a tool router. You decide which external API to call.

          Available tools:
          - "weather.lookup" → args: {{"city":"string","country":"string|null"}}
          - "fx.convert" → args: {{"base":"string","target":"string"}}

          Rules:
          - If user asks about weather or temperature → "weather.lookup"
          - If user asks about exchange rate, currency, USD to INR → "fx.convert"
          - Otherwise return {{"tool_name":"none","args":{{}}}}

          Return ONLY valid JSON:
          {{"tool_name":"...", "args":{{...}}}}

          User: "{user_text}"
          """

    print(query_prompt)

    gemini_model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite-001",
        google_api_key=gemini_api_key,
        temperature=0.3 )

    print("---   Response ---")

    assistant_reply = create_roles_chain(gemini_model).invoke({"input": query_prompt})
    print(f"[Assistant Thought] {assistant_reply}")
    cleaned = assistant_reply.strip()

    if cleaned.startswith("```"):
        cleaned = cleaned.split("```")[1]  # get inside the fence
    cleaned = cleaned.replace("json", "").strip()

    # ------------------------------------------------------------------
    # 2. Convert JSON string → Python dict
    # ------------------------------------------------------------------
    try:
        parsed = json.loads(cleaned)
    except Exception as e:
        print("JSON parse error:", e)
        parsed = {"tool_name": "none", "args": {}}

    return parsed

In [ ]:
def finalize_answer(user_text: str, tool_result: dict) -> str:

    prompt = f"""
      You are a friendly assistant.
      User asked: "{ user_text }"
      Tool result (factual): "{ tool_result }"

      Write a short, friendly answer for the user.
      If tool_result contains an error, explain it briefly.
    """
    gemini_model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite-001",
        google_api_key=gemini_api_key,
        temperature=0.3 )

    print("---   Response ---")
    reply = create_roles_chain(gemini_model).invoke({"input": prompt})
    print(f"[Assistant Thought] {reply}")

    return reply


In [ ]:

def execute_tool(tool_name: str, args: dict) -> dict:
    if tool_name == "weather.lookup":
        return get_weather(args.get("city", ""), args.get("country"))
    if tool_name == "fx.convert":
        return call_exchange_api(args.get("base", "USD"), args.get("target", "INR"))
    if tool_name == "none":
        return {"info": "no tool needed"}
    return {"error": f"unknown tool {tool_name}"}


In [ ]:

def master_agent(user_text: str) -> str:
    route = route_intent(user_text)
    print(route)
    tool_name = route.get("tool_name", "none")
    print(tool_name)
    args = route.get("args", {})
    tool_result = execute_tool(tool_name, args)
    print(tool_result)
    final_answer = finalize_answer(user_text, tool_result)
    return final_answer

In [ ]:
print(master_agent("what is the weather like in new delhi"))
#print(master_agent("what is the usd to inr conversion"))


          You are a tool router. You decide which external API to call.

          Available tools:
          - "weather.lookup" → args: {"city":"string","country":"string|null"}
          - "fx.convert" → args: {"base":"string","target":"string"}

          Rules:
          - If user asks about weather or temperature → "weather.lookup"
          - If user asks about exchange rate, currency, USD to INR → "fx.convert"
          - Otherwise return {"tool_name":"none","args":{}}

          Return ONLY valid JSON:
          {"tool_name":"...", "args":{...}}

          User: "what is the weather like in new delhi"
          
---   Response ---


ResourceExhausted: 429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details.